In [ ]:
import sys
import subprocess
from pybit.unified_trading import HTTP
import boto3
import os
from pathlib import Path

from torch import optim
from torch import nn
import torch
from models.TS.GRUPipeline import GRUFrac
from torch.utils.data import DataLoader, TensorDataset
from models.TS.fracdiff.torch import fdiff

from numpy import dtype

In [1]:

def main(SYMBOL):
    print(SYMBOL)
    session = boto3.session.Session()
    s3 = session.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id='',
        aws_secret_access_key='',
        region_name='ru-central1'
    )
    
    bucket_name = 'test-actions'

    train_file = f'train/all_price_{SYMBOL}.pt'
    
    try:
        s3.download_file(bucket_name, train_file, train_file[6:])
    except:
        return


    train_file = f'train/all_price_{SYMBOL}.pt'

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    all_price = torch.load(f'all_price_{SYMBOL}.pt', weights_only=True).to(torch.float32).to(device)

    mean, std = all_price.detach().mean().item(), all_price.detach().std().item()
    
    x_train = torch.stack([all_price[i:i+20].reshape(20, 1) for i in range(int(all_price.shape[0]*0.8) - 20)]).to(device)
    y_train = torch.tensor([all_price[20+i] for i in range(int(all_price.shape[0]*0.8) - 20)]).reshape(int(all_price.shape[0]*0.8)-20, 1).to(device)
    
    x_test = torch.stack([all_price[i:i+20].reshape(20, 1) for i in range(int(all_price.shape[0]*0.8) - 20, all_price.shape[0] - 20)]).to(device)
    y_test = torch.tensor([all_price[20+i] for i in range(int(all_price.shape[0]*0.8) - 20, all_price.shape[0] - 20)]).reshape(int(all_price.shape[0]*0.2), 1).to(device)
    
    dataset = TensorDataset(x_train, y_train)
    validation = TensorDataset(x_test, y_test)

    batch_size = 128
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(validation, batch_size=128, shuffle=False)

    gru = GRUFrac(input_size=1, hidden_size=40, num_layers=3, output_size=1, mean=mean, scale=std, activation=nn.LeakyReLU())
    gru.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(gru.parameters(), lr=0.001, weight_decay=0.001)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=1.5)
    scheduler2 = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    # Обучение модели
    num_epochs = 500
    patience = 3
    target_loss = float('inf')
    patience_counter = 4

    for epoch in range(num_epochs):
        for i, (batch_x, batch_y) in enumerate(dataloader):
            gru.train()
            outputs = gru(batch_x)
            optimizer.zero_grad()
            loss = criterion(outputs, batch_y-mean)
            loss.backward()
            optimizer.step()
        #if epoch < 7:
        #    scheduler.step()
        if (epoch > 2 and epoch < 12) or (epoch > 20 and epoch < 25):
            scheduler2.step()
        elif (epoch > 12 and epoch < 21):
            scheduler.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        gru.eval()  # Переключаем модель в режим оценки
        val_loss = 0.0
        with torch.no_grad():  # Выключаем вычисление градиентов
            for batch_x, batch_y in val_dataloader:
                outputs = gru(batch_x)
                loss = criterion(outputs, batch_y-mean)
                val_loss += loss.item()

        val_loss /= len(val_dataloader)  # Средняя ошибка на валидации
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss:.4f}')

        # Реализация ранней остановки
        if val_loss < target_loss:
            weights_file = f'weights_{SYMBOL}.pt'
            torch.save(gru.eval().state_dict(), weights_file)
            target_loss = val_loss
            patience_counter = 0  # Сбрасываем счетчик
        else:
            patience_counter += 1  # Увеличиваем счетчик

        # Проверяем, если сохранять, если ошибка стабильна
        if patience_counter >= patience:
            print("Ошибка валидации не уменьшается, остановка обучения.")
            break

    #model_file = f'gru_{SYMBOL}.onnx'
    #torch.onnx.export(gru, x_train, model_file, export_params=True)
    #s3.upload_file(f'gru_{SYMBOL}.onnx', bucket_name, 'model/'+model_file)

    s3.upload_file(weights_file, bucket_name, 'model/'+weights_file)

In [4]:
client = HTTP()

# Получите информацию о рынке
market_data = client.get_tickers(category='spot')

# Отсортируйте токены по объему торгов, чтобы найти самые популярные
sorted_tickers = sorted(filter(lambda x: x.get('symbol').endswith('USDT') and not x.get('symbol').startswith('USD'), market_data.get('result', {}).get('list', [])), key=lambda x: float(x['turnover24h']), reverse=True)

# Выведите топ несколько популярных токенов
top_tickers = sorted_tickers[:10]  # например, топ 5
#for ticker in top_tickers:
#    print(f"Токен: {ticker['symbol']}, Объём за 24ч: {ticker['turnover24h']}")

symbols = [ticker['symbol'] for ticker in top_tickers]

for symbol in symbols:
    main(symbol)